# Algorithmes génétiques #

Vous avez demandé à des élèves de classer leurs choix de sujets de PIE et vous ne savez pas comment faire les groupes de travail ? Les algorithmes génétiques sont là pour vous ! Voici un manuel :

In [1]:
import pandas as pd
import numpy as np

## 1. Récupération des données

La première chose à faire est d'organiser un jeu de données. Admettons donc que pour chaque PIE, il y ait des besoins particuliers de compétences en un certain domaine (parmi une liste de domaines que l'on numérote de 1 à Nd), avec un certain niveau de 1 à 10. Puis admettons par la suite, que chaque élève ait une ou plusieurs compétences, chacune avec un certain niveau noté de 1 à 10. Et enfin, chaque élève a ordonné ses cinq sujets préférés, numérotés entre 1 (pour le plus apprécié)et 5 (pour le moins apprécié).

Une fois ce jeu de données réalisé, nous pouvons le charger :

In [12]:
data_eleves = pd.read_csv('DonneesEleves.csv')

data_eleves.head()

,Eleves,Informatique,Dynamique des Fluides,Mecanique du Vol,Gestion de Projet,Anglais,PIE 1,PIE 2,PIE 3,PIE 4,PIE 5
0,Alban,7,10,3,7,3,1,2,3,4,5
1,Benoit,10,9,10,5,10,2,3,4,5,1
2,Cecile,6,4,2,10,8,3,4,5,1,2
3,David,5,6,10,5,7,4,5,1,2,3
4,Etienne,10,2,1,10,4,5,1,2,3,4


In [13]:
data_PIE = pd.read_csv('DonneesPIE.csv')

data_PIE.head()

,PIE,Informatique,Dynamique des Fluides,Mecanique du Vol,Gestion de Projet,Anglais
0,PIE 1,8,8,10,8,9
1,PIE 2,9,8,9,10,8
2,PIE 3,8,8,10,8,8
3,PIE 4,8,9,8,9,8
4,PIE 5,8,8,9,9,8


In [46]:
#print(data_eleves.get(2,127))
print(data_eleves['Informatique'][1])

10


In [15]:
print(data_eleves.shape, data_PIE.shape)
print(data_PIE.shape[0])

(15, 11) (5, 6)
5


## Initialisation

In [14]:
Ng = 5 #nombre d'élèves par groupe de PIE
Ne = data_eleves.shape[0] # Nombre d'élèves
Npie = data_PIE.shape[0] #Nombre de sujets de PIE
Ndom = data_PIE.shape[1] -1 #Nombre de domaines évalués

n_pop=1000
k_select=250
p_mut=0.01
coef_prof = 1
coef_ch = 1
coef_group = 50
max_iter = 100

pop=[]

#initialisation
for repartition in range(n_pop):
    pop.append(np.random.randint(1, Npie+1, size=Ne))

## 2. Choix d'une fonction à optimiser

La fonction à optimiser est caractéristique des choix de priorité que l'on fait vis-à-vis des différents critères...lequel sera considéré plus important ? Exprimons-la dans cette partie.

In [16]:
def profil(repartition):
    prof = np.zeros((Npie,Ndom))
    
    for e in range(Ne):
        PIE = repartition[e]
        for d in range(Ndom):
            note = data_eleves[e][d+1]
            if note > prof[PIE][d]:
                prof[PIE][d] = note
    
    return prof

def choice(repartition):
    ch = np.zeros(Npie)
    
    for e in range(Ne):
        PIE = repartition[e]
        ch[PIE] += data_eleves[e][Ndom + PIE + 1]
        
    return ch

def nbgroup(repartition):
    nb = np.zeros(Npie)
    
    for e in range(Ne):
        PIE = repartition[e]
        nb[PIE] += 1
        
    return nb


def score_PIE(repartition):
    prof = profil(repartition)
    ch = choice(repartition)
    nb = nbgroup(repartition)
    score = 0
    
    for p in Npie:
        for d in Ndom:
            score += coef_prof * max(0, data_PIE[p][d+1] - prof[p][d])
        
        score += coef_ch * ch[p]
        
        if (nb[p] !=0) or (nb[p] !=5):
            score += coef_group
        
    return score
    

### Decat was here

## 3. Mise en place de l'algorithme

In [17]:
import numpy as np

#retourne les croisements de deux parents
def cross(a1, a2):
    b1 = np.zeros(Ne)
    b2 = np.zeros(Ne)
    for i in range(Ne):
        r = np.random.uniform()
        if(r<0.5):
            b1[i]=a1[i]
            b2[i]=a2[i]
        else:
            b1[i]=a2[i]
            b2[i]=a1[i]
    return b1, b2

In [18]:
#retourne la liste des couples qui vont générer
def select(k, pop):
    n_pop = len(pop)
    l_select = []
    n = 0
    while(n < n_pop-k):
        i_a1 = np.random.randint(0,n_pop-1)
        i_a2 = np.random.randint(0,n_pop-1)
        if i_a2 != i_a1 :
            n += 2 
            l_select.append((pop[i_a1],pop[i_a2]))
    return l_select

In [19]:
#Effectue la mutation d'une repartition (un eleve et un PIE choisis au hasard)
def mutate(repartition):
    i1_mut = np.random.randint(0, len(repartition)-1)
    i2_mut = np.random.randint(0, len(repartition)-1)
    PIE_i1 = repartition[i1_mut]
    PIE_i2 = repartition[i2_mut]
    repartition[i1_mut] = PIE_i2
    repartition[i2_mut] = PIE_i1
    
    return repartition

In [20]:
#Calcul du score d'une repartition
def evaluate(k, pop):
    pop_max = list()
    l_score = []
    
    for i in range(len(pop)):
            l_score.append(score_PIE(pop[i]))
    
    M = max(l_score)
        
    for _ in range(k):
        m = min(l_score)
        i = 0
        while l_score[i] != m:
            i += 1
        pop_max.append(pop[i])
        l_score[i] = M + 1
        
    return pop_max

In [22]:
k_iter = 0
best_score = Ne * 10000
best_repartition = np.zeros(Ne)
while(k_iter < max_iter and best_score > 15):
    new_pop = evaluate(k_select, pop)
    best_repartition = new_pop[0]
    best_score = score_PIE(best_repartition)
    for a1, a2 in select(k_select, pop):
        b1, b2 = cross(a1, a2)
        r1 = np.random.uniform()
        if(r1 < p_mut):
            mutate(b1)
        r2 = np.random.uniform()
        if(r2 < p_mut):
            mutate(b2)
        new_pop.append(b1)
        new_pop.append(b2)
    if(len(new_pop)>n_pop):
        new_pop.pop()
    pop = new_pop
    k_iter += 1
    
print(best_word, k_iter)

KeyError: 0